In [2]:
# !wget https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/bert-bahasa/singlish.txt

In [3]:
with open('singlish.txt') as fopen:
    singlish = fopen.read().split('\n')
    
len(singlish)

19870767

In [9]:
import re
from unidecode import unidecode
import cleaning
from tqdm import tqdm

def preprocessing(string):
    string = unidecode(string)
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    
    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub('[^A-Za-z ]+', ' ', unidecode(string))
    string = re.sub(r'[ ]+', ' ', string).strip()
    
    return string.lower()

def loop(strings):
    for i in tqdm(range(len(strings))):
        strings[i] = preprocessing(strings[i])
    return strings

In [10]:
singlish = cleaning.multiprocessing(singlish, loop, cores = 26)

100%|██████████| 764260/764260 [00:22<00:00, 33767.81it/s]


In [12]:
singlish_words = set(' '.join(singlish).split())
len(singlish_words)

527759

In [13]:
import malaya

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [15]:
singlish_minus = singlish_words - malaya.texts._english_words._english_words
len(singlish_minus)

474399

In [28]:
manglish_vocab = {
    'siasui', 'lah', 'chun', 'kapster', 'ah', 'leh', 'lansi', 'lan si', 'meh',
    'stim', 'maluation', 'kantoi', 'seh', 'yam', 'hor', 'la', 'cha',
    'tao', 'amoi', 'aiya', 'angmor', 'angpau', 'beng', 'chow', 'batam',
    'liao', 'nian', 'buiji', 'hou', 'guo', 'jiang', 'chiu', 'buji',
    'hao', 'kam', 'wan', 'yao', 'cao', 'ciao', 'jin', 'hoseh',
    'jiak', 'ying', 'leybit', 'sibei', 'laobu', 'sia', 'cilok',
    'cibai', 'cb', 'entao', 'gwai', 'kai', 'kongmong', 'kapcai',
    'lanjiao', 'lancau', 'lalazai', 'momantai', 'paikia', 'paiseh',
    'pokai', 'seow', 'sohai', 'sueh', 'tapau', 'wor', 'hor',
    'terrer', 'chop', 'lansi', 'paiseh', 'syok', 'shiok',
    'sibeh', 'kawkaw', 'abuden'
}

In [29]:
from tqdm import tqdm

manglish = []
for s in tqdm(singlish):
    if len(set(s.split()) & manglish_vocab):
        manglish.append(s)

100%|██████████| 19870767/19870767 [00:48<00:00, 411396.50it/s]


In [30]:
len(manglish), len(singlish)

(849593, 19870767)

In [31]:
import json

with open('manglish.json', 'w') as fopen:
    json.dump(manglish, fopen)